In [1]:
import numpy as np
from numpy import asarray
import cv2
from sklearn.model_selection import train_test_split 
import tensorflow

C:\Users\prath\anaconda\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\prath\anaconda\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\prath\anaconda\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\prath\anaconda\envs\py37\lib\site-packages\tensorflow\python\framework

In [11]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization

In [12]:
data = np.load('Garbage_npz_y6.npz')

In [13]:
data.files

['arr_0', 'arr_1', 'arr_2', 'arr_3']

In [14]:
X_train = data['arr_0']
X_test = data['arr_2']
Y_train = data['arr_1']
Y_test = data['arr_3']

In [15]:
X_train=X_train/255
X_test=X_test/255
Y_train = Y_train.T
Y_test = Y_test.T
print(Y_test.shape)
print(Y_train.shape)

(253, 6)
(2274, 6)


In [16]:
TRAIN = 2274
TEST = 253
ROWS = 64
COLS = 64
CH = 3

In [17]:
X_train = X_train.reshape(TRAIN, ROWS, COLS, CH)
X_test = X_test.reshape(TEST, ROWS, COLS, CH)

In [18]:
model=Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(64,64,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
# model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
# model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
# model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(6, activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)      

In [ ]:
history = model.fit(X_train, Y_train, epochs=25, validation_split=0.1)


Train on 2046 samples, validate on 228 samples
Epoch 1/25
2046/2046 [==============================] - 9s 4ms/step - loss: 1.5206 - accuracy: 0.3671 - val_loss: 1.2446 - val_accuracy: 0.5000
Epoch 2/25
2046/2046 [==============================] - 9s 5ms/step - loss: 1.2461 - accuracy: 0.5156 - val_loss: 1.1700 - val_accuracy: 0.5132
Epoch 3/25
2046/2046 [==============================] - 8s 4ms/step - loss: 1.0569 - accuracy: 0.6056 - val_loss: 0.9484 - val_accuracy: 0.6754
Epoch 4/25
2046/2046 [==============================] - 8s 4ms/step - loss: 0.9111 - accuracy: 0.6618 - val_loss: 0.9344 - val_accuracy: 0.6491
Epoch 5/25
2046/2046 [==============================] - 10s 5ms/step - loss: 0.7420 - accuracy: 0.7258 - val_loss: 1.1847 - val_accuracy: 0.5965
Epoch 6/25
2046/2046 [==============================] - 10s 5ms/step - loss: 0.6668 - accuracy: 0.7517 - val_loss: 0.8222 - val_accuracy: 0.7412
Epoch 7/25
2046/2046 [==============================] - 10s 5ms/step - loss: 0.5067 - 

In [ ]:
Y_hat = model.predict(X_train)
train_accuracy = np.mean(np.rint(Y_hat) == Y_train)
print(train_accuracy)

In [ ]:
Y_hat_test = model.predict(X_test)
test_accuracy = np.mean(np.rint(Y_hat_test) == Y_test)
print(test_accuracy)